In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd  /content/yolov5

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install ultralytics

In [ ]:
!chmod +x train.py

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov5x6u.pt')

model.model.nc, model.info()

In [ ]:
def freeze_layers(model, num_freeze):
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]
    for k, v in model.model.named_parameters():
        v.requires_grad = True  # Enable training for all layers initially
        if any(x in k for x in freeze):
            print(f'Freezing layer: {k}')
            v.requires_grad = False  # Freeze the backbone layers

# Freeze the first 10 layers (usually the backbone layers)
freeze_layers(model, num_freeze=10)

In [ ]:
batch_size = 16
epochs = 30
learning_rate = 0.001

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="mmUfdJ0p2CzLuX1Kjk2b")
project = rf.workspace("ayush-nndar").project("fire_aug1")
version = project.version(1)
dataset = version.download("yolov5")


In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /kaggle/working/yolov5/fire_aug1-1/data.yaml

# roboflow:
#   license: CC BY 4.0
#   project: fire_aug1
#   url: https://universe.roboflow.com/ayush-nndar/fire_aug1/dataset/1
#   version: 1
#   workspace: ayush-nndar

names: [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light",
    "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
    "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch",
    "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse", "remote", "keyboard", "cell phone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
    "hair drier", "toothbrush", "fire"
]

nc: 81 # number of classes

test: /kaggle/working/yolov5/fire_aug1-1/test/images
train: /kaggle/working/yolov5/fire_aug1-1/train/images
val: /kaggle/working/yolov5/fire_aug1-1/valid/images


In [ ]:
!cat /kaggle/working/yolov5/fire_aug1-1/data.yaml

In [ ]:
%%writetemplate /kaggle/working/yolov5/models/yolov5x.yaml
# Ultralytics YOLOv5 🚀, AGPL-3.0 license

# Parameters
nc: 81 # number of classes
depth_multiple: 1.33 # model depth multiple
width_multiple: 1.25 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32


# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]


In [ ]:
!cat /kaggle/working/yolov5/models/yolov5x.yaml

In [ ]:
file_path = "/kaggle/working/yolov5/train.py"

# Read the content of the file
with open(file_path, 'r') as file:
    file_content = file.read()

# Replace the deprecated function call
updated_content = file_content.replace(
    "torch.cuda.amp.autocast(amp)",
    "torch.amp.autocast('cuda', dtype=torch.float16 if amp else torch.float32)"
)

# Write the updated content back to the file
with open(file_path, 'w') as file:
    file.write(updated_content)

print("File updated successfully.")

In [ ]:
!cat train.py | grep "torch.amp.autocast"

In [ ]:
!python train.py --batch {batch_size} --img 416 --epochs {epochs} --data fire_aug1-1/data.yaml --weights yolov5x6u.pt --cfg models/yolov5x.yaml